## **Project 1: Financial Assistant**

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from openai.types.beta import Assistant
import yfinance as yf
import ccxt
from openai.types.beta.threads import Message
import time

In [3]:
import warnings

# Ignore only DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [5]:
_

True

In [6]:
# Get the key
api_key = os.getenv("OPENAI_API_KEY")

# Check and print whether it's loaded
if api_key:
    print("🔐 OpenAI Key Loaded:", bool(api_key))  # True or False
if not api_key:
    raise EnvironmentError("❌ OPENAI_API_KEY not found in .env file!")

🔐 OpenAI Key Loaded: True


In [7]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

#### **Defining custom function**

In [8]:
def get_stock_summary(ticker: str):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "name": info.get("shortName"),
        "price": info.get("regularMarketPrice"),
        "market_cap": info.get("marketCap"),
        "pe_ratio": info.get("trailingPE"),
        "summary": info.get("longBusinessSummary", ""),
        "location": info.get("city", "") + ", " + info.get("state", "")
    }


def normalize_symbol(symbol_input: str) -> str:
    cleaned = symbol_input.strip().upper().replace("-", "/").replace("_", "/")
    parts = cleaned.split("/")

    if len(parts) == 1:
        base = parts[0]
        quote = "USDT"
    elif len(parts) == 2:
        base, quote = parts
    else:
        raise ValueError(f"Invalid symbol format: {symbol_input}")

    return f"{base}/{quote}"  # MEXC expects 'BTC/USDT'

# Main crypto fetch function
def get_crypto_summary(symbol: str):
    try:
        exchange = ccxt.mexc()
        exchange.load_markets()

        normalized_symbol = normalize_symbol(symbol)  # e.g., 'BTC/USDT'

        if normalized_symbol not in exchange.symbols:
            return {"error": f"Symbol {normalized_symbol} not found on MEXC."}

        ticker = exchange.fetch_ticker(normalized_symbol)
        market_data = exchange.market(normalized_symbol)

        return {
            "symbol": normalized_symbol.replace("/", "_"),  # Display style: BTC_USDT
            "price": ticker.get("last"),
            "high_24h": ticker.get("high"),
            "low_24h": ticker.get("low"),
            "volume_24h": ticker.get("quoteVolume"),
            "change_pct": ticker.get("percentage"),
            "base": market_data.get("base"),
            "quote": market_data.get("quote"),
            "info": "Live data from MEXC (via ccxt)"
        }

    except Exception as e:
        return {"error": str(e)}

# Entry point for Assistant tool
def fetch_crypto_data(user_symbol: str):
    try:
        return get_crypto_summary(user_symbol)
    except Exception as e:
        return {"error": str(e)}


In [9]:
available_functions = {
    "fetch_crypto_data": fetch_crypto_data,
    "get_stock_summary": get_stock_summary
}

## **Tools Schemas**

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_summary",
            "description": "Get financial summary for a publicly traded stock (e.g., AAPL, TSLA, GOOGL). Only for companies listed on stock exchanges.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Stock ticker symbol (e.g., TSLA, AAPL)"
                    }
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fetch_crypto_data",
            "description": "Get market data for cryptocurrencies like BTC, ETH, DOGE using MEXC API. Use this for any digital assets or crypto coins.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_symbol": {
                        "type": "string",
                        "description": "Crypto symbol like 'BTC', 'ETH', or 'BTC/USDT'"
                    }
                },
                "required": ["user_symbol"]
            }
        }
    }
]


## **Create Assistants**

In [11]:
assistant : Assistant = client.beta.assistants.create(
    name="AI Financial Analyst",
    instructions="You are a professional financial analyst. Use functions as needed to analyze assets.",
    tools=tools,
    model="gpt-3.5-turbo-1106"
)

## **Create threads**

In [12]:
from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()

## **Helper Functions**

In [13]:
def add_message_to_thread(client, thread_id, content: str):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def start_run(client, thread_id, assistant_id):
    return client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )



## **Run and Polling**

In [14]:
def run_and_poll(client, thread_id, run_id, available_functions):
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Run Status: {run_status.status}")

        if run_status.status == "requires_action":
            print("Assistant requires action (tool calls)...")
            tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                print(f"Tool Call: {function_name}({function_args})")

                if function_name in available_functions:
                    try:
                        result = available_functions[function_name](**function_args)
                        if not isinstance(result, str):
                            result = json.dumps(result)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": result
                        })
                    except Exception as e:
                        print(f"Error in {function_name}: {e}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": json.dumps({"error": str(e)})
                        })

            print("Submitting tool outputs...")
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run_id,
                tool_outputs=tool_outputs
            )

        elif run_status.status == "completed":
            print("Run completed. Fetching final assistant message...\n")

            # Filter only messages created in this specific run
            messages = client.beta.threads.messages.list(thread_id=thread_id)
            for message in reversed(messages.data):
                if message.role == "assistant" and message.run_id == run_id:
                    reply = message.content[0].text.value
                    print("Assistant Reply:\n", reply)
                    return reply
            return None

        elif run_status.status in ["in_progress", "queued"]:
            time.sleep(2)

        elif run_status.status == "failed":
            print("Run failed.")
            return None

        else:
            print(f"Unexpected status: {run_status.status}")
            return None


In [15]:
# Send message + run assistant + handle tools
def send_and_run_message(client, thread_id, assistant_id, content, available_functions):
    print(f"\n💬 New User Message: {content}")

    # Step 1: Add message
    add_message_to_thread(client, thread_id, content)

    # Step 2: Start run
    run = start_run(client, thread_id, assistant_id)

    # Step 3: Poll and return final reply
    return run_and_poll(client, thread_id, run.id, available_functions)

In [16]:
tsla_performance = send_and_run_message(
    client=client,
    thread_id=thread.id,
    assistant_id=assistant.id,
    content="Analyze TSLA performance.",
    available_functions=available_functions
)


💬 New User Message: Analyze TSLA performance.
Run Status: queued
Run Status: in_progress
Run Status: requires_action
Assistant requires action (tool calls)...
Tool Call: get_stock_summary({'ticker': 'TSLA'})
Submitting tool outputs...
Run Status: queued
Run Status: in_progress
Run Status: completed
Run completed. Fetching final assistant message...

Assistant Reply:
 Tesla, Inc. (TSLA) is a company that designs, develops, manufactures, leases, and sells electric vehicles, as well as energy generation and storage systems. Here are some key details about TSLA:

- Stock Name: Tesla, Inc.
- Stock Price: $329.65
- Market Cap: $1,061,789,433,856
- P/E Ratio: 188.37
- Location: Austin, TX
- Summary: TSLA operates in two segments: Automotive and Energy Generation and Storage. The Automotive segment offers electric vehicles, automotive regulatory credits, vehicle services, and retail merchandise. The Energy Generation and Storage segment engages in the design, manufacture, installation, sale, 

In [17]:
BTC_performance = send_and_run_message(
    client=client,
    thread_id=thread.id,
    assistant_id=assistant.id,
    content="Analyze BTC performance.",
    available_functions=available_functions
)



💬 New User Message: Analyze BTC performance.
Run Status: queued
Run Status: requires_action
Assistant requires action (tool calls)...
Tool Call: fetch_crypto_data({'user_symbol': 'BTC'})
Submitting tool outputs...
Run Status: queued
Run Status: in_progress
Run Status: completed
Run completed. Fetching final assistant message...

Assistant Reply:
 Bitcoin (BTC) is currently priced at $117,935.25. In the last 24 hours, the price of BTC reached a high of $119,642.85 and a low of $116,467.02. The 24-hour trading volume for BTC is $583,598,939.64, and the price change percentage is -0.14%.

Bitcoin's performance in the crypto market indicates fluctuations in price and significant trading volume, making it a key asset for many investors and traders.


In [18]:
BTC_performance = send_and_run_message(
    client=client,
    thread_id=thread.id,
    assistant_id=assistant.id,
    content="Analyze ETH performance.",
    available_functions=available_functions
)



💬 New User Message: Analyze ETH performance.
Run Status: queued
Run Status: queued
Run Status: queued
Run Status: requires_action
Assistant requires action (tool calls)...
Tool Call: fetch_crypto_data({'user_symbol': 'ETH'})
Submitting tool outputs...
Run Status: queued
Run Status: in_progress
Run Status: completed
Run completed. Fetching final assistant message...

Assistant Reply:
 Ethereum (ETH) is currently priced at $3,786.10. In the last 24 hours, the price of ETH reached a high of $3,836.53 and a low of $3,682.47. The 24-hour trading volume for ETH is $546,179,429.07, and the price change percentage is 1.08%.

ETH has shown positive price movement in the last 24 hours, with significant trading volume and price fluctuations. This indicates active interest and engagement in the Ethereum market.
